In [4]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime

def find_nearest_raven_eye(lat, lon, channel="BDF"):
    client = Client("IRIS")
    
    # 1. Search for stations within 0.5 degrees (~55km) of the target
    print(f"Scanning for 'Eyes' near {lat}, {lon}...")
    try:
        inventory = client.get_stations(
            latitude=lat, 
            longitude=lon, 
            maxradius=2.0, 
            level="channel",
            channel=channel,
            starttime=UTCDateTime.now() - 86400 # Must be active in last 24h
        )

        print(inventory)
        
        # 2. Extract the first available station found
        network = inventory[0].code
        station = inventory[0][0].code
        
        print(f"Found Eye: {network}.{station}")
        return network, station
        
    except Exception as e:
        print(f"No active {channel} stations nearby: {e}")
        return None, None

# Example: Near the Pentagon
target_lat, target_lon = 38.8719, -77.0563
net, sta = find_nearest_raven_eye(target_lat, target_lon)

Scanning for 'Eyes' near 38.8719, -77.0563...
Inventory created at 2026-01-26T12:17:04.793700Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.52
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2026-01-...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (3):
			IU, LD, N4
		Stations (12):
			IU.SSPA (Standing Stone, Pennsylvania)
			LD.GEDE (Greenville, DE, USA)
			LD.WADE (Warrington Farm, Harbeson, DE)
			N4.N58A (Sunbury, PA, USA)
			N4.P57A (Homestead Farm, Martinsburg, WV, USA)
			N4.P61A (Hammonton, NJ, USA)
			N4.Q56A (Snyder Ridge, Maysville, WV, USA)
			N4.R58B (Mineral, VA, USA)
			N4.R61A (Willards, MD, USA)
			N4.S57A (Dark Hallow, Roseland, VA, USA)
			N4.S61A (Accomac, VA, USA)
			N4.T59A (Double 'B' Farms, VA, USA)
		Channels (12):
			IU.SSPA.32.BDF, LD.GEDE.EP.BDF, LD.WADE.EP.BDF, N4.N58A.32.BDF, 
			N4.P57A.32.BDF, N4.P61A.32.BDF, N4.Q56A.32.BDF, N4.R58B.32.BDF, 
			N4.R61A.32.BDF, N4.S57A.32.BDF, N4.S61A.32.BDF, N4.T59A.32.B

In [1]:
import nexradaws
import datetime

conn = nexradaws.NexradAwsInterface()
# Example: KOKX (New York/Long Island)
radar_id = 'KOKX'
start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)

avail_scans = conn.get_avail_scans(start.year, start.month, start.day, radar_id)
# Grab the most recent Volume Coverage Pattern (VCP)
results = conn.download(avail_scans[-1], './data')
results

/tmp/ipykernel_84259/1564378900.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  start = datetime.datetime.utcnow() - datetime.timedelta(hours=1)


Downloaded KOKX20260127_102227_V06
1 out of 1 files downloaded...0 errors


In [1]:
import requests
import pandas as pd
from datetime import datetime

# --- CONFIGURATION ---
# Replace with your actual eBird API Key
EBIRD_API_KEY = "h0m6j8ep3cv2"

# Define Target Locations (Latitude, Longitude)
# Zone A: The Base (High Security Zone)
ZONE_A_COORDS = {'lat': 34.296, 'lng': -116.142} # Example coords
ZONE_A_RADIUS_KM = 2

# Zone B: The Suburb (Civilian area surrounding Zone A)
ZONE_B_COORDS = {'lat': 34.350, 'lng': -116.180} # Example coords (nearby)
ZONE_B_RADIUS_KM = 5

# Target Family: Corvidae (Crows, Ravens, Jays, Magpies)
TARGET_FAMILY_CODE = "corvid" 
# Note: eBird doesn't filter by family in this endpoint nicely, 
# so we will filter by common name keywords in the script.
TARGET_KEYWORDS = ["Crow", "Raven", "Corvus"]

# iNaturalist Configuration
INAT_BASE_URL = "https://api.inaturalist.org/v1/observations"
ANOMALY_TERMS = ["Bird Strike", "Injured", "Dead", "Collision"]

# --- HELPER FUNCTIONS ---

def get_ebird_data(lat, lng, dist_km, api_key):
    """
    Fetches recent observations from eBird 2.0 API.
    """
    url = "https://api.ebird.org/v2/data/obs/geo/recent"
    headers = {'X-eBirdApiToken': api_key}
    params = {
        'lat': lat,
        'lng': lng,
        'dist': dist_km,
        'back': 7,  # Look back 7 days
        'fmt': 'json'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching eBird data: {e}")
        return []

def filter_corvids(data, keywords):
    """
    Filters observation list for target keywords (Crows/Ravens).
    Returns a count of individual birds observed.
    """
    count = 0
    sightings = []
    
    for obs in data:
        com_name = obs.get('comName', '')
        # Check if any keyword matches the common name
        if any(k.lower() in com_name.lower() for k in keywords):
            # eBird 'howMany' can be None if count is not reported (marked as 'X')
            # We assume 1 for presence if 'X', otherwise use the number.
            qty = obs.get('howMany')
            if qty is None: 
                qty = 1 
            
            count += qty
            sightings.append({
                'species': com_name,
                'count': qty,
                'loc': obs.get('locName'),
                'date': obs.get('obsDt')
            })
            
    return count, sightings

def get_inaturalist_anomalies(lat, lng, radius_km, terms):
    """
    Queries iNaturalist for proxy terms (Bird Strike, Injured).
    """
    anomalies = []
    
    for term in terms:
        params = {
            'q': term,
            'lat': lat,
            'lng': lng,
            'radius': radius_km, 
            'per_page': 5 # Keep it light
        }
        
        try:
            response = requests.get(INAT_BASE_URL, params=params)
            response.raise_for_status()
            results = response.json().get('results', [])
            
            for res in results:
                anomalies.append({
                    'term_match': term,
                    'species': res.get('species_guess'),
                    'description': res.get('description'),
                    'url': res.get('uri')
                })
        except Exception as e:
            print(f"Error fetching iNaturalist data for '{term}': {e}")
            
    return anomalies

# --- MAIN EXECUTION ---

def run_eye_iii():
    print(f"--- EYE III: THE CORVID SHADOW ---")
    print(f"Time: {datetime.now()}")
    
    # 1. Zone A Analysis
    print(f"\n[Zone A] Scanning Base (Radius: {ZONE_A_RADIUS_KM}km)...")
    data_a = get_ebird_data(ZONE_A_COORDS['lat'], ZONE_A_COORDS['lng'], ZONE_A_RADIUS_KM, EBIRD_API_KEY)
    count_a, details_a = filter_corvids(data_a, TARGET_KEYWORDS)
    print(f"   > Raw Observations: {len(data_a)}")
    print(f"   > Corvid Count: {count_a}")

    # 2. Zone B Analysis
    print(f"\n[Zone B] Scanning Suburbs (Radius: {ZONE_B_RADIUS_KM}km)...")
    data_b = get_ebird_data(ZONE_B_COORDS['lat'], ZONE_B_COORDS['lng'], ZONE_B_RADIUS_KM, EBIRD_API_KEY)
    count_b, details_b = filter_corvids(data_b, TARGET_KEYWORDS)
    print(f"   > Raw Observations: {len(data_b)}")
    print(f"   > Corvid Count: {count_b}")

    # 3. Calculate KPI: Species Ratio
    print(f"\n[KPI Calculation]")
    if count_a == 0:
        ratio = 0
        print("   > Ratio: Undefined (Zone A count is 0). No base activity detected.")
    else:
        ratio = count_b / count_a
        print(f"   > Species Ratio (Suburbs / Base): {ratio:.2f}")

    # Interpretation
    if ratio > 5.0:
        print("   > STATUS: HIGH DISPLACEMENT. Ravens are heavily favoring the suburbs.")
    elif ratio < 1.0 and count_a > 0:
        print("   > STATUS: CONCENTRATED. Ravens are clustering at the Base.")
    else:
        print("   > STATUS: NOMINAL. Normal distribution.")

    # 4. Secondary Source Check (iNaturalist)
    print(f"\n[Secondary Source] Checking iNaturalist for Anomalies in Zone B...")
    anomalies = get_inaturalist_anomalies(ZONE_B_COORDS['lat'], ZONE_B_COORDS['lng'], ZONE_B_RADIUS_KM, ANOMALY_TERMS)
    
    if anomalies:
        print(f"   > ALERT: {len(anomalies)} potential proxy events found.")
        df = pd.DataFrame(anomalies)
        print(df[['term_match', 'species', 'url']].to_string(index=False))
    else:
        print("   > No 'Bird Strike' or 'Injured' reports found in the immediate window.")

if __name__ == "__main__":
    run_eye_iii()

--- EYE III: THE CORVID SHADOW ---
Time: 2026-01-31 14:30:19.600584

[Zone A] Scanning Base (Radius: 2km)...
   > Raw Observations: 0
   > Corvid Count: 0

[Zone B] Scanning Suburbs (Radius: 5km)...
   > Raw Observations: 0
   > Corvid Count: 0

[KPI Calculation]
   > Ratio: Undefined (Zone A count is 0). No base activity detected.
   > STATUS: NOMINAL. Normal distribution.

[Secondary Source] Checking iNaturalist for Anomalies in Zone B...
   > No 'Bird Strike' or 'Injured' reports found in the immediate window.


In [2]:
import requests
import json

def test_ebird_key():
    # --- CONFIGURATION ---
    # The key you provided
    API_KEY = "h0m6j8ep3cv2" 
    
    # Test Location: Yarkon Park, Tel Aviv (High probability of sightings)
    TEST_LAT = 32.090
    TEST_LNG = 34.795
    
    # Endpoint: Recent observations in a geometric area
    url = "https://api.ebird.org/v2/data/obs/geo/recent"
    
    # Headers must include the token
    headers = {
        "X-eBirdApiToken": API_KEY
    }
    
    # Query Parameters
    params = {
        "lat": TEST_LAT,
        "lng": TEST_LNG,
        "back": 7,      # Look back 7 days
        "dist": 10,     # 10km radius
        "maxResults": 5 # Just get 5 to prove it works
    }

    print(f"Testing API Key: {API_KEY}...")
    print(f"Targeting: {TEST_LAT}, {TEST_LNG} (Tel Aviv)")
    
    try:
        response = requests.get(url, headers=headers, params=params)
        
        # Check specific HTTP codes
        if response.status_code == 200:
            data = response.json()
            print("\n✅ SUCCESS: The API Key is active.")
            print(f"   > Retrieved {len(data)} observations.\n")
            
            if len(data) > 0:
                print("--- Sample Data ---")
                for bird in data:
                    name = bird.get('comName', 'Unknown')
                    count = bird.get('howMany', 1)
                    loc = bird.get('locName', 'Unknown')
                    print(f"   • {name} (Count: {count}) @ {loc}")
            else:
                print("   (Key worked, but no birds found in this radius. Try a wider radius.)")
                
        elif response.status_code == 403:
            print("\n❌ FAILURE: Access Forbidden (403).")
            print("   > The key might be incorrect or not yet activated.")
            
        else:
            print(f"\n⚠️ ISSUE: Server returned status {response.status_code}")
            print(f"   > Response: {response.text}")

    except Exception as e:
        print(f"\n❌ ERROR: Script execution failed.")
        print(f"   > {e}")

if __name__ == "__main__":
    test_ebird_key()

Testing API Key: h0m6j8ep3cv2...
Targeting: 32.09, 34.795 (Tel Aviv)

✅ SUCCESS: The API Key is active.
   > Retrieved 5 observations.

--- Sample Data ---
   • Rock Pigeon (Count: 2) @ פארקפה מרכז פארק הרצליה
   • Hooded Crow (Count: 3) @ פארקפה מרכז פארק הרצליה
   • Great Tit (Count: 1) @ פארקפה מרכז פארק הרצליה
   • Common Myna (Count: 6) @ פארקפה מרכז פארק הרצליה
   • House Sparrow (Count: 4) @ פארקפה מרכז פארק הרצליה


In [3]:
import requests
import pandas as pd
from datetime import datetime
import time

# --- CONFIGURATION ---
# Your verified eBird API Key
EBIRD_API_KEY = "h0m6j8ep3cv2"

# Define Your Locations (Latitude, Longitude)
# UPDATE THESE: Currently set to two distinct points near Tel Aviv for demonstration.
# Zone A: The Base (e.g., restricted facility)
ZONE_A_COORDS = {'lat': 32.100, 'lng': 34.800} 
ZONE_A_RADIUS_KM = 2

# Zone B: The Suburb (e.g., civilian area 5-10km away)
ZONE_B_COORDS = {'lat': 32.080, 'lng': 34.770} 
ZONE_B_RADIUS_KM = 5

# Target Filters
# "Hooded Crow" was in your test data. "Corvus" catches Ravens.
TARGET_KEYWORDS = ["Crow", "Raven", "Corvus", "Jackal", "Jay", "Magpie"]

# iNaturalist Configuration (Secondary Source)
INAT_BASE_URL = "https://api.inaturalist.org/v1/observations"
ANOMALY_TERMS = ["Bird Strike", "Injured", "Dead", "Collision"]

# --- HELPER FUNCTIONS ---

def get_ebird_data(lat, lng, dist_km, api_key):
    """
    Fetches recent observations from eBird 2.0 API.
    """
    url = "https://api.ebird.org/v2/data/obs/geo/recent"
    headers = {'X-eBirdApiToken': api_key}
    params = {
        'lat': lat,
        'lng': lng,
        'dist': dist_km,
        'back': 7,  # Look back 7 days
        'fmt': 'json'
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"   [!] Error fetching eBird data: {e}")
        return []

def filter_corvids(data, keywords):
    """
    Filters observation list for target keywords (Crows/Ravens).
    Returns a count of individual birds observed.
    """
    count = 0
    sightings = []
    
    for obs in data:
        com_name = obs.get('comName', '')
        # Check if any keyword matches the common name
        if any(k.lower() in com_name.lower() for k in keywords):
            # eBird 'howMany' is None if presence is marked 'X'. We count that as 1.
            qty = obs.get('howMany')
            if qty is None: 
                qty = 1 
            
            count += qty
            sightings.append({
                'species': com_name,
                'count': qty,
                'loc': obs.get('locName'),
                'date': obs.get('obsDt')
            })
            
    return count, sightings

def get_inaturalist_anomalies(lat, lng, radius_km, terms):
    """
    Queries iNaturalist for proxy terms (Bird Strike, Injured).
    """
    anomalies = []
    
    for term in terms:
        params = {
            'q': term,
            'lat': lat,
            'lng': lng,
            'radius': radius_km, 
            'per_page': 3  # Limit results to keep it fast
        }
        
        try:
            # Adding a user-agent is good practice for iNaturalist
            headers = {'User-Agent': 'ThreeEyedRaven_Project/1.0'}
            response = requests.get(INAT_BASE_URL, params=params, headers=headers)
            response.raise_for_status()
            results = response.json().get('results', [])
            
            for res in results:
                anomalies.append({
                    'term_match': term,
                    'species': res.get('species_guess'),
                    'description': res.get('description'),
                    'url': res.get('uri')
                })
            time.sleep(1) # Respect rate limits
        except Exception as e:
            print(f"   [!] Error fetching iNaturalist data for '{term}': {e}")
            
    return anomalies

# --- MAIN EXECUTION ---

def run_eye_iii():
    print(f"--- EYE III: THE CORVID SHADOW ---")
    print(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"----------------------------------")
    
    # 1. Zone A Analysis
    print(f"\n[Zone A] Scanning Base Coordinates ({ZONE_A_COORDS['lat']}, {ZONE_A_COORDS['lng']})...")
    data_a = get_ebird_data(ZONE_A_COORDS['lat'], ZONE_A_COORDS['lng'], ZONE_A_RADIUS_KM, EBIRD_API_KEY)
    count_a, details_a = filter_corvids(data_a, TARGET_KEYWORDS)
    print(f"   > Total Observations: {len(data_a)}")
    print(f"   > Corvid Abundance:   {count_a}")
    if details_a:
        print(f"   > Species found: {', '.join(set([d['species'] for d in details_a]))}")

    # 2. Zone B Analysis
    print(f"\n[Zone B] Scanning Suburb Coordinates ({ZONE_B_COORDS['lat']}, {ZONE_B_COORDS['lng']})...")
    data_b = get_ebird_data(ZONE_B_COORDS['lat'], ZONE_B_COORDS['lng'], ZONE_B_RADIUS_KM, EBIRD_API_KEY)
    count_b, details_b = filter_corvids(data_b, TARGET_KEYWORDS)
    print(f"   > Total Observations: {len(data_b)}")
    print(f"   > Corvid Abundance:   {count_b}")
    if details_b:
        print(f"   > Species found: {', '.join(set([d['species'] for d in details_b]))}")

    # 3. Calculate KPI: Species Ratio
    print(f"\n[KPI Calculation]")
    
    # Avoid division by zero
    safe_count_a = count_a if count_a > 0 else 1
    
    ratio = count_b / safe_count_a
    print(f"   > Displacement Ratio (Suburb/Base): {ratio:.2f}")

    # Interpretation of the Index
    if count_a == 0 and count_b == 0:
        print("   > STATUS: NULL. No Corvids detected in either zone.")
    elif ratio > 3.0:
        print("   > STATUS: HIGH DISPLACEMENT. (Alert Level: RED)")
        print("     Significant movement towards civilian infrastructure detected.")
    elif ratio < 0.5:
        print("   > STATUS: CONCENTRATED. (Alert Level: YELLOW)")
        print("     Unusual gathering at the Base.")
    else:
        print("   > STATUS: NOMINAL. Distribution is balanced.")

    # 4. Secondary Source Check (iNaturalist)
    print(f"\n[Secondary Source] Checking iNaturalist for Anomalies...")
    anomalies = get_inaturalist_anomalies(ZONE_B_COORDS['lat'], ZONE_B_COORDS['lng'], ZONE_B_RADIUS_KM, ANOMALY_TERMS)
    
    if anomalies:
        print(f"   > ALERT: {len(anomalies)} potential proxy events found!")
        df = pd.DataFrame(anomalies)
        # Adjust display for terminal
        pd.set_option('display.max_colwidth', 50)
        print(df[['term_match', 'species', 'url']].to_string(index=False))
    else:
        print("   > No 'Bird Strike' or 'Injured' reports found in the immediate window.")

if __name__ == "__main__":
    run_eye_iii()

--- EYE III: THE CORVID SHADOW ---
Date: 2026-01-31 14:36:03
----------------------------------

[Zone A] Scanning Base Coordinates (32.1, 34.8)...
   > Total Observations: 62
   > Corvid Abundance:   5
   > Species found: Hooded Crow, Black-crowned Night Heron, Eurasian Jay

[Zone B] Scanning Suburb Coordinates (32.08, 34.77)...
   > Total Observations: 65
   > Corvid Abundance:   4
   > Species found: Hooded Crow, Black-crowned Night Heron, Eurasian Jay

[KPI Calculation]
   > Displacement Ratio (Suburb/Base): 0.80
   > STATUS: NOMINAL. Distribution is balanced.

[Secondary Source] Checking iNaturalist for Anomalies...
   > ALERT: 5 potential proxy events found!
term_match           species                                                url
   Injured Egyptian Rousette https://www.inaturalist.org/observations/255600350
   Injured       Common Myna https://www.inaturalist.org/observations/157677403
      Dead              None https://www.inaturalist.org/observations/328181658
      D